Spark RDD feladatok



In [38]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
sc = SparkContext.getOrCreate()


In [39]:
x = "MWűGkyösdwinko Qau fSJpDaTrykv!w"

rdd = sc.parallelize([x[i:i+2] for i in range(0, len(x), 2)])
res = rdd.map(lambda x: x[0]).collect()

print("".join(res))

Működik a Spark!


1. feladat (2 pont)
A vasarlasok.txt egy kiskereskedelmi áruházban megvásárolt termékeket tartalmazzák. Minden sor
egy vásárló kosarában lévő termékeket sorolja fel, vesszővel elválasztva. Határozd meg azt a három
terméket, amelyekből a legtöbbet vásárolták és add meg hányat. Figyelj oda arra, hogy az
adathalmazban kis és nagy betűk is előfordulhatnak. Azaz előfordulhat a "rizs" és a "Rizs" termék is.
Ezeket azonos termékeknek kell tekintened.

Egy lehetséges kimenet: ('kenyer', 108), ('cukor', 68), ('tej', 64)

In [55]:
lines = sc.textFile('vasarlasok.txt')\
.flatMap(lambda l: l.split(','))\
.map(lambda l: (l.lower(), 1))\
.reduceByKey(lambda a,b: a + b)\
.sortBy(lambda x: x[1], False)\
.take(3)

lines

[('kenyer', 108), ('cukor', 68), ('tej', 64)]

2. feladat (2 pont)

Az online_retail_data.csv egy webáruház eladásait tartalmazza. Határozd meg, hogy melyik terméket
melyik termékkel vásárolják gyakran együtt Franciaországban („France”). Azaz melyek a leggyakrabban
előforduló termékpárok. Add meg a párok előfordulásának a számát és a 3 legtöbbször előfordulót ad
vissza.

Egy lehetséges kimenet: [(('POSTAGE', 'RABBIT NIGHT LIGHT'), 130), (('POSTAGE', 'RED TOADSTOOL
LED NIGHT LIGHT'), 124), (('PLASTERS IN TIN CIRCUS PARADE ', 'POSTAGE'), 116)]

In [58]:
# mintamegoldás

retailFile = 'online_retail_data.csv'
retailRDD = sc.textFile(retailFile)

def createPairs(x):
    pairs = []
    for i in list(x[1]):
        for j in list(x[1]):
            if i<j:
                pairs.append((i,j))
            elif j<i:
                pairs.append((j,i))
    return pairs

ret2 = retailRDD.filter(lambda line : not line.startswith("InvoiceNo")).filter(lambda line: line.split(",")[-1] == "France")
res = ret2.map(lambda line: (line.split(",")[0],line.split(",")[2]))\
.groupByKey().map(createPairs).flatMap(lambda x:x).map(lambda x: (x,1))\
.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], False).take(5)
print(res)

[(('POSTAGE', 'RABBIT NIGHT LIGHT'), 130), (('POSTAGE', 'RED TOADSTOOL LED NIGHT LIGHT'), 124), (('PLASTERS IN TIN CIRCUS PARADE ', 'POSTAGE'), 116), (('POSTAGE', 'ROUND SNACK BOXES SET OF4 WOODLAND '), 116), (('PLASTERS IN TIN WOODLAND ANIMALS', 'POSTAGE'), 108)]


Spark DataFrame feladatok

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

In [2]:
menu = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("menu.csv")

category = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("menuCategory.csv")

menu.show()
category.show()

+--------+--------------------+------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
|Category|                Item|Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+--------+--------------------+------------+--------+-----------------+---------+-------------------------+-------------+---------------

3. feladat (1 pont)
Melyek azok az ételek, amelyek meghaladják az ajánlott napi zsír bevitelt? (Total Fat (% Daily Value))

Elvárt oszlopok: [Item]

In [7]:
from pyspark.sql.functions import col
menu.where(col("Total Fat (% Daily Value)") > 100)\
.select("Item")\
.show()

+--------------------+
|                Item|
+--------------------+
|Chicken McNuggets...|
+--------------------+



4. feladat (1 pont)
Melyik ételnek van a maximális Sugars értéke?

Elvárt oszlopok: [Item, Sugars]

In [12]:
menu.select("Item", "Sugars")\
.orderBy(col("Sugars").desc())\
.limit(1)\
.show()

+--------------------+------+
|                Item|Sugars|
+--------------------+------+
|McFlurry with M&M...|   128|
+--------------------+------+



5. feladat (1 pont)
Hány elem van kategóriánként? Rendezzük csökkenő sorrendbe és adjuk meg a kategóriák nevét is.

Elvárt oszlopok: [Name, Cnt]

In [21]:
menu.alias("M")\
.groupBy(col("Category"))\
.agg(count(col("*")).alias("Cnt"))\
.join(category.alias("C"), col("C.ID") == col("M.Category"), "inner")\
.select("Name","Cnt")\
.orderBy(col("Cnt").desc())\
.show(9)

+------------------+---+
|              Name|Cnt|
+------------------+---+
|      Coffee & Tea| 95|
|         Breakfast| 42|
|Smoothies & Shakes| 28|
|    Chicken & Fish| 27|
|         Beverages| 27|
|       Beef & Pork| 15|
|    Snacks & Sides| 13|
|          Desserts|  7|
|            Salads|  6|
+------------------+---+



6. feladat (1 pont)
Átlagosan mennyi kalóriát tartalmaznak az egyes kategóriák? Adjuk meg a kategória nevét és
rendezzük átlag alapján csökkenő sorrendbe. A 'Coffee and Tea' és a 'Beverages' kategóriákat ne
vegyük figyelembe.

In [32]:
menu.alias("M")\
.groupBy(col("Category"))\
.agg(avg(col("Calories")).alias("avgCal"))\
.join(category.where((col("Name") != "Coffee & Tea") & (col("Name") != "Beverages")).alias("C"), col("C.ID") == col("M.Category"), "inner")\
.select("Name","avgCal")\
.orderBy(col("avgCal").desc())\
.show(9)

+------------------+------------------+
|              Name|               avg|
+------------------+------------------+
|    Chicken & Fish| 552.9629629629629|
|Smoothies & Shakes| 531.4285714285714|
|         Breakfast| 526.6666666666666|
|       Beef & Pork|             494.0|
|            Salads|             270.0|
|    Snacks & Sides|245.76923076923077|
|          Desserts|222.14285714285714|
+------------------+------------------+

